## Pipeline Ações negociadas na bolsa

### Questão de negocio

Ação do S&P 500

### Entendimento do negocio

https://www.kaggle.com/camnugent/sandp500/

● Date - no formato yy-mm-dd

● Open - Preço da ação na abertura do mercado no dia, em dólares. 

● High - Maior preço alcançado naquele dia.

● Low - Menor preço alcançado naquele dia.

● Close - Preço da ação no fechamento do mercado no dia.

● Volume - Número de ações vendidas / compradas.

● Name - O nome da ação.

### 1 - Importar bibliotecas
### 2 - Coletar dados
### 3 - Descrição dos dados
### 4 - Análise de dados

### 1 - Importar bibliotecas

In [1]:
import findspark
findspark.init()

In [15]:
import pyspark.sql.functions as f

In [3]:
from pyspark.sql import SparkSession

### 2 - Coletar dados

In [4]:
spark = SparkSession.builder.appName('SparkTrabalhoPraticoXP.com').getOrCreate()

In [5]:
df = spark.read.csv('C:/XP_DataScience/Modulo2/archive/all_stocks_5yr.csv',
                    header= True, inferSchema= True)

### 3 - Descrição dos dados

In [6]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



In [7]:
df.describe().show()

+-------+----------+----------------+-----------------+-----------------+-----------------+-----------------+------+
|summary|      date|            open|             high|              low|            close|           volume|  Name|
+-------+----------+----------------+-----------------+-----------------+-----------------+-----------------+------+
|  count|    619040|          619029|           619032|           619032|           619040|           619040|619040|
|   mean|      null|83.0233343145474|83.77831069347276|82.25609641375338|83.04376276476519|4321823.395568945|  null|
| stddev|      null|97.3787690433237|98.20751890446375|96.50742105809033|97.38974800165782|8693609.511967566|  null|
|    min|2013-02-08|            1.62|             1.69|              1.5|             1.59|                0|     A|
|    max|2018-02-07|          2044.0|          2067.99|          2035.11|           2049.0|        618237630|   ZTS|
+-------+----------+----------------+-----------------+---------

### 4 - Análise de dados

### 1 - Quantos registros há na planilha?

#### 619040

### 2 - Quantos registros há na planilha para a ação da Apple (AAPL)?

###### count #operaçoes de registros

In [ ]:
# operações de registros
# where() e filter()

De toda forma, ambos os métodos de filtragem são aplicados à DataFrames e estão intimamente associados à operações lógicas, 
permitindo assim a criação implícita de booleanos que são avaliados e utilizados como regra principal para seleção 
de registros

In [8]:
df.where(df['Name'] == 'AAPL').count()

1259

### 3 - Quantas empresas distintas têm registros nessa planilha? 

#### count #distinct #consultas

In [ ]:
# criando consultas
# select()

In [9]:
df.select('Name').distinct().count()

505

### 4 - Com qual frequência o preço de uma ação no fechamento é maior do que o preço na abertura?

#### count where > 

#### operação de registro
#### where operação

In [11]:
open_close = df.where(df['close'] > df['open']).count()

In [12]:
round((open_close/df.count()) * 100, 2)

51.53

### 5 - Qual o maior valor das ações da Apple (AAPL) na história?

#### describe maior operação de registro

In [13]:
df.where(df['name'] == 'AAPL').describe().show()

+-------+----------+------------------+------------------+------------------+------------------+-------------------+----+
|summary|      date|              open|              high|               low|             close|             volume|Name|
+-------+----------+------------------+------------------+------------------+------------------+-------------------+----+
|  count|      1259|              1259|              1259|              1259|              1259|               1259|1259|
|   mean|      null|109.05542891183475|109.95111834789516|108.14158888006372|109.06669849086583|5.404789973550437E7|null|
| stddev|      null| 30.54922002458826|30.686186407983914|30.376223585852472|30.556811676964696|  3.3468353335784E7|null|
|    min|2013-02-08|           55.4242|           57.0857|           55.0142|           55.7899|           11475922|AAPL|
|    max|2018-02-07|            179.37|             180.1|            178.25|            179.26|          266833581|AAPL|
+-------+----------+----

### 6 - Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar a ação de maior desvio-padrão.

#### operação de registro ordenar maior sort operação de registro categorica e numerica groupby e agg

In [39]:
df.groupBy('Name').agg(f.stddev('close').alias('Desvio padrão')).sort('Desvio padrão', ascending= False).show(5)

+-----+------------------+
| Name|     Desvio padrão|
+-----+------------------+
| PCLN|  320.533473018748|
| AMZN|  282.500395033522|
|GOOGL|187.57389185479704|
| GOOG|166.42052902768182|
|  MTD|131.39721989584996|
+-----+------------------+
only showing top 5 rows



### 7 - Qual o dia com maior volume total de negociação da bolsa?

In [40]:
df.groupBy('date').agg(f.sum('volume').alias('Volume total')).sort('Volume total', ascending= False).show(5)

+----------+------------+
|      date|Volume total|
+----------+------------+
|2015-08-24|  4607945196|
|2016-06-24|  4367393052|
|2015-12-18|  4124454411|
|2016-01-20|  4087629753|
|2018-02-06|  4072080890|
+----------+------------+
only showing top 5 rows



### 8 - Qual a ação mais negociada da bolsa, em volume de transações?

In [43]:
df.groupBy('Name').agg(f.sum('volume').alias('Ação mais negociada em volume')).sort('Ação mais negociada em volume', ascending= False).show(5)

+----+-----------------------------+
|Name|Ação mais negociada em volume|
+----+-----------------------------+
| BAC|                 117884953591|
|AAPL|                  68046305767|
|  GE|                  53023644686|
|   F|                  43388129992|
|  FB|                  43258314899|
+----+-----------------------------+
only showing top 5 rows



### 9 - Quantas ações começam com a letra “A”?

In [17]:
df.where(df['Name'].startswith('A')).select('Name').distinct().count()

59

### 10 - Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?

In [18]:
high_close = df.where(df['high'] == df['close']).count()

In [19]:
round((high_close/df.count()) * 100, 2)

1.2

### 11 - Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta?

In [20]:
df_apple = df.where((df['Name'] == 'AAPL') & (df['close'] > df['open']))
df_apple.show(5)

+----------+-------+-------+-------+-------+---------+----+
|      date|   open|   high|    low|  close|   volume|Name|
+----------+-------+-------+-------+-------+---------+----+
|2013-02-08|67.7142|68.4014|66.8928|67.8542|158168416|AAPL|
|2013-02-11|68.0714|69.2771|67.6071|68.5614|129029425|AAPL|
|2013-02-14|66.3599|67.3771|66.2885|66.6556| 88809154|AAPL|
|2013-02-21|63.7142|64.1671|63.2599|63.7228|111596821|AAPL|
|2013-02-22|64.1785|64.5142|63.7999|64.4014| 82583823|AAPL|
+----------+-------+-------+-------+-------+---------+----+
only showing top 5 rows



In [21]:
df_apple = df_apple.withColumn('difference_value', df['close'] - df['open'])
df_apple.show()

+----------+-------+-------+-------+-------+---------+----+--------------------+
|      date|   open|   high|    low|  close|   volume|Name|    difference_value|
+----------+-------+-------+-------+-------+---------+----+--------------------+
|2013-02-08|67.7142|68.4014|66.8928|67.8542|158168416|AAPL| 0.14000000000000057|
|2013-02-11|68.0714|69.2771|67.6071|68.5614|129029425|AAPL|  0.4900000000000091|
|2013-02-14|66.3599|67.3771|66.2885|66.6556| 88809154|AAPL| 0.29570000000001073|
|2013-02-21|63.7142|64.1671|63.2599|63.7228|111596821|AAPL|0.008600000000001273|
|2013-02-22|64.1785|64.5142|63.7999|64.4014| 82583823|AAPL| 0.22289999999999566|
|2013-02-26|63.4028|64.5056|62.5228|64.1385|125096657|AAPL|  0.7356999999999942|
|2013-03-05|60.2114|62.1699|60.1071|61.5919|159298020|AAPL|   1.380500000000005|
|2013-03-07|60.6428|61.7157|60.1514|61.5117|116992841|AAPL|  0.8688999999999965|
|2013-03-08|61.3999|62.2042|61.2299|61.6742| 97854442|AAPL| 0.27429999999999666|
|2013-03-11|61.3928|62.7157|

In [22]:
df_apple.sort('difference_value', ascending= False).show(5)

+----------+------+------+------+------+---------+----+------------------+
|      date|  open|  high|   low| close|   volume|Name|  difference_value|
+----------+------+------+------+------+---------+----+------------------+
|2015-08-24| 94.87| 108.8|  92.0|103.12|162206292|AAPL|              8.25|
|2018-02-06|154.83|163.72| 154.0|163.03| 68243838|AAPL| 8.199999999999989|
|2017-05-08|149.03| 153.7|149.03|153.01| 48752413|AAPL|3.9799999999999898|
|2014-12-09|110.19| 114.3|109.35|114.12| 60208036|AAPL| 3.930000000000007|
|2017-10-27|159.29| 163.6| 158.7|163.05| 44454160|AAPL|3.7600000000000193|
+----------+------+------+------+------+---------+----+------------------+
only showing top 5 rows



### 12 - Em média, qual o volume diário de transações das ações da AAPL?

In [26]:
df.where(df['Name'] == 'AAPL').select(f.mean('volume').alias('Média do volume diário AAPL')).show()

+---------------------------+
|Média do volume diário AAPL|
+---------------------------+
|        5.404789973550437E7|
+---------------------------+



### 13 - Quantas ações tem 1, 2, 3, 4 e 5 caracteres em seu nome, respectivamente?

In [ ]:
1 caractere

In [27]:
df.where(f.length('Name') == 1).select('Name').distinct().count()

10

In [28]:
df.where(f.length('Name') == 2).select('Name').distinct().count()

50

In [29]:
df.where(f.length('Name') == 3).select('Name').distinct().count()

323

In [30]:
df.where(f.length('Name') == 4).select('Name').distinct().count()

117

In [31]:
df.where(f.length('Name') == 5).select('Name').distinct().count()

5

### 14 - Qual a ação menos negociada da bolsa, em volume de transações?

In [37]:
df.groupBy('Name').agg(f.sum('volume').alias('Ação menos negociada em volume')).sort('Ação menos negociada em volume', ascending = True).show(5)

+----+------------------------------+
|Name|Ação menos negociada em volume|
+----+------------------------------+
|APTV|                      92947779|
| BHF|                     144878392|
| MTD|                     214489754|
| AZO|                     397585127|
|WLTW|                     414734590|
+----+------------------------------+
only showing top 5 rows

